In [1]:
import pandas as pd
import os
import requests
from requests.exceptions import ConnectionError
from time import sleep
import json
import ydata_profiling as yp
import sqlalchemy
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib_inline
import seaborn as sns

In [2]:
df = pd.read_csv('ormedru.csv', sep=';')

In [3]:
df

,Регион таргетинга,Показы,Клики,Расход (руб.),Конверсия (%),Цена цели (руб.),Конверсии
0,Абакан,2352,33,"706,44","6,06","353,22",2
1,Азов,488,7,"155,68",-,-,-
2,Актау,47526,200,"2381,51","0,5","2381,51",1
3,Актау и Мангистауская область (более точно рег...,4111,11,"126,04",-,-,-
4,Александров,754,12,"162,88",-,-,-
...,...,...,...,...,...,...,...
710,Ямало-Ненецкий автономный округ (более точно р...,1305,34,"856,39",-,-,-
711,Ярославль,7534,165,"2753,01","1,21","1376,51",2
712,Ярославская область (более точно регион не опр...,2733,52,"992,71",-,-,-
713,Ярцево,312,6,"232,41",-,-,-


In [4]:
df.rename(columns=
{
          'Регион таргетинга' : 'TargetingLocationName',
          'Конверсия (%)' : 'Conversion_Rate',
          'Цена цели (руб.)' : 'CPA',
          'Конверсии' : 'Conversions'          
},
inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TargetingLocationName  715 non-null    object 
 1   Показы                 715 non-null    int64  
 2   Клики                  715 non-null    int64  
 3   Расход (руб.)          715 non-null    object 
 4   Conversion_Rate        715 non-null    float64
 5   CPA                    715 non-null    float64
 6   Conversions            715 non-null    int32  
dtypes: float64(2), int32(1), int64(2), object(2)
memory usage: 36.4+ KB


In [7]:
df.CPA = df.CPA.replace('-', '0.0').str.replace(',', '.').astype(float)
df.Conversion_Rate = df.Conversion_Rate.replace('-', '0.0').str.replace(',', '.').astype(float)
df.Conversions = df.Conversions.replace('-', '0').astype(int)

In [9]:
rfm = df.groupby("TargetingLocationName").agg({
    "Conversion_Rate": "mean",           # C (чем выше, тем лучше)
    "Conversions": "sum",  # F (чем больше, тем лучше)
    "CPA": "mean"                        # M (чем ниже, тем лучше)
}).rename(columns={
    "Conversion_Rate": "Conversion_Rate",
    "Conversions": "Frequency",
    "CPA": "Monetary"
})

In [11]:
# Min-Max нормализация для Conversion Rate (0-1)
rfm.CR_Norm = (rfm.Conversion_Rate - rfm.Conversion_Rate.min()) / (rfm.Conversion_Rate.max() - rfm.Conversion_Rate.min())

# Min-Max нормализация для Frequency (0-1)
rfm.Freq_Norm = (rfm.Frequency - rfm.Frequency.min()) / (rfm.Frequency.max() - rfm.Frequency.min())

# Инвертированная нормализация для CPA (чтобы 1 = лучший)
rfm.CPA_Norm = 1 - (rfm.Monetary - rfm.Monetary.min()) / (rfm.Monetary.max() - rfm.Monetary.min())

# Итоговый балл
rfm.Total_Score = rfm.CR_Norm + rfm.Freq_Norm + rfm.CPA_Norm
rfm.Total_Score_Rank = rfm.Total_Score.rank(ascending=False)

C:\Temp\ipykernel_14876\166362216.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  rfm.CR_Norm = (rfm.Conversion_Rate - rfm.Conversion_Rate.min()) / (rfm.Conversion_Rate.max() - rfm.Conversion_Rate.min())
C:\Temp\ipykernel_14876\166362216.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  rfm.Freq_Norm = (rfm.Frequency - rfm.Frequency.min()) / (rfm.Frequency.max() - rfm.Frequency.min())
C:\Temp\ipykernel_14876\166362216.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  rfm.CPA_Norm = 1 - (rfm.Monetary - rfm.Monetary.min()) / (rfm.Monetary.max() - rfm.Monetary.min())
C:\Temp\ipykernel_14876\166362216.py:11: UserWarning: Pandas doe

In [12]:
rfm.Total_Score

TargetingLocationName
Абакан                                                               1.070448
Азов                                                                 1.000000
Акмолинская область (более точно регион не определён)                1.111069
Актау                                                                0.681253
Актау и Мангистауская область (более точно регион не определён)      1.000000
                                                                       ...   
Ямало-Ненецкий автономный округ (более точно регион не определён)    1.000000
Ярославль                                                            0.869412
Ярославская область (более точно регион не определён)                1.000000
Ярцево                                                               1.000000
Ярцевский район (более точно регион не определён)                    1.000000
Length: 711, dtype: float64

In [13]:
# Разбиваем Total_Score на 5 квантилей
rfm["Quantile_Group"] = pd.qcut(
    rfm["Total_Score"],
    q=3,
    labels=[1, 2, 3]  # 1 — низкий балл, 3 — высокий
)

# Проверяем распределение
print(rfm["Quantile_Group"].value_counts().sort_index())

KeyError: 'Total_Score'

In [65]:
rfm

,Conversion_Rate,Frequency,Monetary
TargetingLocationName,,,
Абакан,6.06,2,353.22
Азов,0.00,0,0.00
Акмолинская область (более точно регион не определён),3.42,4,322.89
Актау,0.50,1,2381.51
Актау и Мангистауская область (более точно регион не определён),0.00,0,0.00
...,...,...,...
Ямало-Ненецкий автономный округ (более точно регион не определён),0.00,0,0.00
Ярославль,1.21,2,1376.51
Ярославская область (более точно регион не определён),0.00,0,0.00


In [68]:
quartiles = rfm.quantile(q=[0.25,0.50,0.75])
print(quartiles, type(quartiles))

      Conversion_Rate  Frequency  Monetary
0.25              0.0        0.0       0.0
0.50              0.0        0.0       0.0
0.75              0.0        0.0       0.0 <class 'pandas.core.frame.DataFrame'>


In [69]:
quartiles=quartiles.to_dict()
quartiles

{'Conversion_Rate': {0.25: 0.0, 0.5: 0.0, 0.75: 0.0},
 'Frequency': {0.25: 0.0, 0.5: 0.0, 0.75: 0.0},
 'Monetary': {0.25: 0.0, 0.5: 0.0, 0.75: 0.0}}

In [71]:
#создаем функцию для определения сегмента по метрике Conversion_Rate (C) и Conversions (F)
#чем меньше значение, тем хуже (присваеваем 1 класс)
#чем выше значение этих метрик, тем лучше (присваем 4 класс)


def CF_class(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
       return 3
    else:
        return 4

In [72]:
#создаем функцию для определения сегмента по метрике CPA (M)
#чем меньше значение, тем лучше (присваеваем 4 класс)
#чем выше значение, тем хуже (присваем 1 класс)

def M_class(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [73]:
#применяем написанные функции к датафрейму rfm для присвоения классов

#копируем датафрейм с другим именем 
rfm_seg = rfm

#создаем новые столбцы, где будут указаны классы для каждого клиента по R,F,M метрикам
rfm_seg['C'] = rfm_seg['Conversion_Rate'].apply(CF_class, args=('Conversion_Rate',quartiles,))
rfm_seg['F'] = rfm_seg['Frequency'].apply(CF_class, args=('Frequency',quartiles,))
rfm_seg['M'] = rfm_seg['Monetary'].apply(M_class, args=('Monetary',quartiles,))

In [74]:
rfm_seg

,Conversion_Rate,Frequency,Monetary,C,F,M
TargetingLocationName,,,,,,
Абакан,6.06,2,353.22,4,4,1
Азов,0.00,0,0.00,1,1,4
Акмолинская область (более точно регион не определён),3.42,4,322.89,4,4,1
Актау,0.50,1,2381.51,4,4,1
Актау и Мангистауская область (более точно регион не определён),0.00,0,0.00,1,1,4
...,...,...,...,...,...,...
Ямало-Ненецкий автономный округ (более точно регион не определён),0.00,0,0.00,1,1,4
Ярославль,1.21,2,1376.51,4,4,1
Ярославская область (более точно регион не определён),0.00,0,0.00,1,1,4


In [75]:
#создаем подключение к базе Postgres на удаленном сервере

HOST = os.environ['HOST']
PORT = os.environ['PORT']
DATABASE = os.environ['DATABASE']
LOGIN = os.environ['LOGIN']
PASSWORD = os.environ['PASSWORD']

engine = sqlalchemy.create_engine(f'postgresql://{LOGIN}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')

In [77]:
#загружаем датафрейм в базу данных

table_name = 'rfm'
schema_name = 'ormed'

rfm_seg.to_sql(con=engine, name = table_name, schema=schema_name, index=False, if_exists='replace')

711